## ETL
#### El dataset provisto por el Observatorio de movilidad y seguridad vial, (OMSV), contiene información sobre accidentes de tránsito. Específicamente se compone de dos hojas dentro de un formato de libro Excel, una sobre los hechos del accidente y otra con datos sobre las víctimas de tales accidentes.
#### Tras haber explorado la estructura de los y características generales en el notebook "1 Análisis inicial", debemos limpiar, organizar y cargar los datos para hacer el análisis multivariado pertinente en el EDA, BI y cumplir los KPIs propuestos.

In [46]:
# Importamos las librerías necesarias para trabajar con nuestro archivo
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [47]:
# Leemos el archivo y sus hojas
df_excel_homicidios = pd.ExcelFile("Data/homicidios.xlsx")
df_hechos = pd.read_excel(df_excel_homicidios, "HECHOS")
df_victimas = pd.read_excel(df_excel_homicidios, "VICTIMAS")

In [48]:
# Visualizamos la hoja "HECHOS" del libro "homicidios"
df_hechos.head(2)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [49]:
# Visualizamos información sobre la hoja
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

#### Algunas de las columnas de la hoja "hechos" contienen información que se encuentra agrupada en otras columnas, por tanto, decidimos eliminar información redundante.

In [50]:
df_hechos = df_hechos.drop(columns=["AAAA", "MM", "DD", "HH", "Dirección Normalizada", "Calle", "Altura", "Cruce", "XY (CABA)"])
df_hechos.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS


In [51]:
# Visualizamos información sobre la hoja
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               696 non-null    object        
 1   N_VICTIMAS       696 non-null    int64         
 2   FECHA            696 non-null    datetime64[ns]
 3   HORA             696 non-null    object        
 4   LUGAR_DEL_HECHO  696 non-null    object        
 5   TIPO_DE_CALLE    696 non-null    object        
 6   COMUNA           696 non-null    int64         
 7   pos x            696 non-null    object        
 8   pos y            696 non-null    object        
 9   PARTICIPANTES    696 non-null    object        
 10  VICTIMA          696 non-null    object        
 11  ACUSADO          696 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 65.4+ KB


#### Nos disponemos a analizar la información de la hoja "victimas" para ver si resulta conveniente agrupar las dos hojas en un solo dataframe que disponibilizaremos para el EDA.

In [52]:
# Visualizamos la hoja "VICTIMAS" del libro "homicidios"
df_victimas.head(2)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### Esta hoja presenta la misma característica que la anterior, tener columnas agrupadas en otras, por lo que el procedimiento puede ser el mismo.

In [53]:
df_victimas = df_victimas.drop(columns=["AAAA", "MM", "DD", "FECHA"])
df_victimas.head(2)

,ID_hecho,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### Las columna "ID" de la hoja "hechos" coincide con la columna "ID_hecho" de la hoja "victimas". Por ende, debemos hacer el merge a partir de dichas columnas.

In [54]:
df_accidentes = pd.merge(df_hechos, df_victimas, left_on="ID", right_on="ID_hecho")
df_accidentes.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-0001,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00


#### En este punto notamos que los valores nulos están registrados como "SD", equivalente a "Sin Datos". Los reemplazaremos por valores nulos para no sesgar la visualización.

In [55]:
def reemplazar_SD(df):
    for i in range(len(df)):
        for col in df.columns:
            if df.loc[i, col] == "SD":
                df.loc[i, col] = pd.NaT
    return df

reemplazar_SD(df_accidentes)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ID_hecho,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,2016-0001,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,2016-0002,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,07:00:00,AV ENTRE RIOS 2034,AVENIDA,1,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,2016-0003,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,00:00:00,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,8,-58.46503904,-34.68092974,MOTO-SD,MOTO,NaT,2016-0004,CONDUCTOR,MOTO,MASCULINO,18,NaT
4,2016-0005,1,2016-01-21,05:20:00,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,1,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,2016-0005,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17:10:00,AV. RIESTRA Y MOM,AVENIDA,7,-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO,2021-0093,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
713,2021-0094,1,2021-12-20,01:10:00,AU DELLEPIANE Y LACARRA,AUTOPISTA,9,-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO,2021-0094,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
714,2021-0095,1,2021-12-30,00:43:00,AV. GAONA Y TERRADA,AVENIDA,11,-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS,2021-0095,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00
715,2021-0096,1,2021-12-15,10:30:00,AV. EVA PERON 4071,AVENIDA,9,-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS,2021-0096,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20 00:00:00


In [56]:
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   N_VICTIMAS           717 non-null    int64         
 2   FECHA                717 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      716 non-null    object        
 5   TIPO_DE_CALLE        717 non-null    object        
 6   COMUNA               717 non-null    int64         
 7   pos x                717 non-null    object        
 8   pos y                717 non-null    object        
 9   PARTICIPANTES        717 non-null    object        
 10  VICTIMA_x            708 non-null    object        
 11  ACUSADO              694 non-null    object        
 12  ID_hecho             717 non-null    object        
 13  ROL                  706 non-null  

#### Después de hacer el merged, vemos que hay columnas con información repetida, seguimos con la eliminación.

In [57]:
df_accidentes = df_accidentes.drop(columns=["ID_hecho", "VICTIMA_y"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   717 non-null    object        
 1   N_VICTIMAS           717 non-null    int64         
 2   FECHA                717 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      716 non-null    object        
 5   TIPO_DE_CALLE        717 non-null    object        
 6   COMUNA               717 non-null    int64         
 7   pos x                717 non-null    object        
 8   pos y                717 non-null    object        
 9   PARTICIPANTES        717 non-null    object        
 10  VICTIMA_x            708 non-null    object        
 11  ACUSADO              694 non-null    object        
 12  ROL                  706 non-null    object        
 13  SEXO                 711 non-null  

#### En este punto podemos empezar a tratar valores faltantes en las columnas.

In [58]:
# Eliminamos un registro del que no se conoce la hora del suceso
df_accidentes = df_accidentes.dropna(subset=["HORA"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 716 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   716 non-null    object        
 1   N_VICTIMAS           716 non-null    int64         
 2   FECHA                716 non-null    datetime64[ns]
 3   HORA                 716 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        716 non-null    object        
 6   COMUNA               716 non-null    int64         
 7   pos x                716 non-null    object        
 8   pos y                716 non-null    object        
 9   PARTICIPANTES        716 non-null    object        
 10  VICTIMA_x            707 non-null    object        
 11  ACUSADO              693 non-null    object        
 12  ROL                  705 non-null    object        
 13  SEXO                 710 non-null    obj

#### Eliminamos un registro que no tiene lugar del hecho.

In [59]:
df_accidentes = df_accidentes.dropna(subset=["LUGAR_DEL_HECHO"])
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            706 non-null    object        
 11  ACUSADO              693 non-null    object        
 12  ROL                  704 non-null    object        
 13  SEXO                 710 non-null    obj

#### En las columnas "VICTIMA_X", "ACUSADO" y "ROL" tenemos valores nulos a los que podemos imputarles el valor "No identificado", tratando de conervar la información del registro.

In [60]:
# Completamos los valores nulos de las columnas "VICTIMA", "ACUSADO" y "ROL" por "No identificado"
df_accidentes["VICTIMA_x"].fillna("No identificado", inplace=True)
df_accidentes["ACUSADO"].fillna("No identificado", inplace=True)
df_accidentes["ROL"].fillna("No identificado", inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  ROL                  715 non-null    object        
 13  SEXO                 710 non-null    obj

#### A los valores nulos de la columna "SEXO" se les puede imputar el valor "No especificado", para conservar los registros para aquellas estadísticas que no requieran dichos valores.

In [61]:
# Rellenamos los valores nulos de la columna
df_accidentes["SEXO"].fillna("No especificado", inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  ROL                  715 non-null    object        
 13  SEXO                 715 non-null    obj

#### Los valores nulos de la columna "EDAD" pueden suplirse con la media de todas las edades presentes en la columna.

In [62]:
# Convertimos a tipo de dato numérico la columna, para después rellenar
df_accidentes["EDAD"] = pd.to_numeric(df_accidentes["EDAD"], errors="coerce")

# Calculamos la media
media_edad = df_accidentes["EDAD"].dropna().mean()

# Hacemos la imputación
df_accidentes["EDAD"].fillna(media_edad, inplace=True)

# Cambiamos el tipo de dato int
df_accidentes["EDAD"] = df_accidentes["EDAD"].astype(int)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  ROL                  715 non-null    object        
 13  SEXO                 715 non-null    obj

#### Los valores de la columna "FECHA_FALLECIMIENTO" que son nulos, se deben a que las víctimas fallecieron en el mismo instante que ocurrió el accidente, por tanto, podemos cambiar el formato de dicha columna y rellenar los valore faltantes con los de la columna "FECHA".

In [63]:
# Primero cambiamos los formatos a fecha estandar
df_accidentes["FECHA_FALLECIMIENTO"] = pd.to_datetime(df_accidentes["FECHA_FALLECIMIENTO"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df_accidentes["FECHA_FALLECIMIENTO"] = df_accidentes["FECHA_FALLECIMIENTO"].dt.strftime("%Y-%m-%d")

# Rellenamos los valores nulos de "FECHA_FALLECIMIENTO"
df_accidentes["FECHA_FALLECIMIENTO"].fillna(df_accidentes["FECHA"], inplace=True)
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  ROL                  715 non-null    object        
 13  SEXO                 715 non-null    obj

#### Ahora debemos organizar la estructura de las direcciones de la columna "LUGAR_DEL_HECHO".

In [64]:
# Reemplazamos las comas de la columna por espacios vacios, para después reorganizarlas
df_accidentes["LUGAR_DEL_HECHO"] = df_accidentes["LUGAR_DEL_HECHO"].str.replace(',', '')

In [65]:
# Definimos una función para reorganizar la estructura de los valores de la columna
def reorganizar_direcc(address):
    words = address.split()
    if "Av." in words:
        words.remove("Av.")
        words = ["Av."] + words
    return " ".join(words)

# Aplicamos la función a la columna y visualizamos
df_accidentes["LUGAR_DEL_HECHO"] = df_accidentes["LUGAR_DEL_HECHO"].apply(reorganizar_direcc)
df_accidentes.head(2)

,ID,N_VICTIMAS,FECHA,HORA,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,VICTIMA_x,ACUSADO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,04:00:00,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,8,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,01:15:00,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,9,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,CONDUCTOR,MASCULINO,70,2016-01-02


#### Por último, podemos estandarizar los valores de la columna "HORA".

In [66]:
# Reemplazamos aquellos valores que tienen el formato incorrecto
formato_incorrecto = pd.to_datetime(df_accidentes["HORA"], errors="coerce").isnull()
df_accidentes.loc[~formato_incorrecto, "HORA"] = pd.to_datetime(df_accidentes.loc[~formato_incorrecto, "HORA"]).dt.strftime('%H:%M:%S')
df_accidentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 716
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   715 non-null    object        
 1   N_VICTIMAS           715 non-null    int64         
 2   FECHA                715 non-null    datetime64[ns]
 3   HORA                 715 non-null    object        
 4   LUGAR_DEL_HECHO      715 non-null    object        
 5   TIPO_DE_CALLE        715 non-null    object        
 6   COMUNA               715 non-null    int64         
 7   pos x                715 non-null    object        
 8   pos y                715 non-null    object        
 9   PARTICIPANTES        715 non-null    object        
 10  VICTIMA_x            715 non-null    object        
 11  ACUSADO              715 non-null    object        
 12  ROL                  715 non-null    object        
 13  SEXO                 715 non-null    obj

#### En este punto tenemos un dataframe limpio para hacer parte del análisis multivariado en el EDA y para satisfacer los KPIs propuestos. Lo disponibilizamos.

In [67]:
df_accidentes.to_csv("Data/df_accidentes.csv")